# 심층 신경망 구현

In [3]:
import tensorflow as tf
import numpy as np
# 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델
# 신경망의 레이어를 여러 개로 구성 -> 딥러닝 구성

# [털, 날개]
x_data = np.array( [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# [기타, 포유류, 조류]
y_data = np.array([
[1, 0, 0], # 기타
[0, 1, 0], # 포유류
[0, 0, 1], # 조류
[1, 0, 0],
[1, 0, 0],
[0, 0, 1]
])

####################
# 신경망 모델 구성
####################

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런 개수] -> [2, 10] 으로 정함
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 개수, 분류 개수] -> [10, 3] 으로 정함
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 개수로 설정
# b1 은 히든 레이어의 뉴런 개수로, b2 는 최종 결과값 즉, 분류 개수인 3으로 설정
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

# 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)


# 최종적인 아웃풋을 계산
# 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력 값을 만듦
# 출력층에 활성화 함수를 적용하지 않음 (활성화 함수 적용 여부는 경험적, 실험적 요소임)
model = tf.add(tf.matmul(L1, W2), b2)


# 텐서플로우에서 기본적으로 제공되는 크로스 엔트로피 함수를 이용해
# 복잡한 수식을 사용하지 않고도 최적화를 위한 비용 함수를 다음처럼 간단하게 적용
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

# 최적화 함수로 AdamOptimzer를 사용 경사하강기법보다 보편적으로 성능이 좋다고 알려져 있음
train_op = optimizer.minimize(cost)


#################
# 신경망 모델 학습
#################

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    

##########################
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
##########################
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))
sess.close()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

10 0.9137149
20 0.7338696
30 0.595743
40 0.4817841
50 0.3921443
60 0.32215577
70 0.2657809
80 0.22067703
90 0.18694271
100 0.16116025
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
